In [4]:
%run "../../../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

Setup Complete. Use CLIENT to access Weaviate.


In [5]:
print(CLIENT.get_meta())

{'hostname': 'http://[::]:8080', 'modules': {'generative-openai': {'documentationHref': 'https://beta.openai.com/docs/api-reference/completions', 'name': 'Generative Search - OpenAI'}, 'qna-openai': {'documentationHref': 'https://beta.openai.com/docs/api-reference/completions', 'name': 'OpenAI Question & Answering Module'}, 'text2vec-transformers': {'model': {'_name_or_path': './models/model', 'add_cross_attention': False, 'architectures': ['BertModel'], 'attention_probs_dropout_prob': 0.1, 'bad_words_ids': None, 'begin_suppress_tokens': None, 'bos_token_id': None, 'chunk_size_feed_forward': 0, 'classifier_dropout': None, 'cross_attention_hidden_size': None, 'decoder_start_token_id': None, 'diversity_penalty': 0, 'do_sample': False, 'early_stopping': False, 'encoder_no_repeat_ngram_size': 0, 'eos_token_id': None, 'exponential_decay_length_penalty': None, 'finetuning_task': None, 'forced_bos_token_id': None, 'forced_eos_token_id': None, 'gradient_checkpointing': False, 'hidden_act': 'ge

In [4]:
where_filter = {
  "path": ["source"],
  "operator": "Equal",
  "valueText": "032944da-8e34-4812-98d5-859b927d24b4"
}

response = (
  CLIENT.query
  .get("Work", "title")
  .with_where(where_filter)
  .do()
)

jprint(response)

<IPython.core.display.JSON object>

In [6]:
import json

data_object = CLIENT.data_object.get_by_id(
    '032944da-8e34-4812-98d5-859b927d24b4',
    class_name='Work',
)

print(json.dumps(data_object, indent=2))

{
  "class": "Work",
  "creationTimeUnix": 1689298020505,
  "id": "032944da-8e34-4812-98d5-859b927d24b4",
  "lastUpdateTimeUnix": 1689298020505,
  "properties": {
    "accession_number": "BFMF_B26_F06_280",
    "api_model": "Work",
    "collection": "Berkeley Folk Music Festival",
    "contributor": [
      "Olivier, Barry, 1935-"
    ],
    "create_date": "2021-03-15T15:42:27.515587Z",
    "date_created": [
      "March 31, 1970"
    ],
    "date_created_edtf": [
      "1970-03-31"
    ],
    "description": [
      "Typed letter from Barry Olivier to Faith Petric regarding a Slim Critchlow record and Olivier's career change. Dated March 31, 1970."
    ],
    "genre": [
      "letters (correspondence)"
    ],
    "identifier_descriptive": [
      "MS 63"
    ],
    "language": [
      "English"
    ],
    "library_unit": "Charles Deering McCormick Library of Special Collections",
    "physical_description_size": [
      "11 inches (height) x 8.5 inches (width)"
    ],
    "preservation

In [12]:
print(os.getenv("AZURE_OPENAI_DEPLOYMENT_ID"))

None


In [13]:
class_obj =  {
  "class": "Textract",
  "vectorizer": "text2vec-transformers",
  "moduleConfig": {
      "text2vec-transformers": {
          "vectorizeClassName": True,
          "resourceName": os.getenv("AZURE_OPENAI_RESOURCE_NAME"),
          "deploymentId": os.getenv("AZURE_OPENAI_DEPLOYMENT_ID"),
      }
  },
  "properties": [
      {"dataType": ["text"], "name": "full_text"},   
      {
          "name": "hasWork",
          "dataType": ["Work"],
          "description": "The parent work",
      },       
  ]
}


CLIENT.schema.create_class(class_obj)  # returns null on success



In [7]:
import json
response = CLIENT.schema.get()

print(json.dumps(response, indent=2))

{
  "classes": [
    {
      "class": "Work",
      "invertedIndexConfig": {
        "bm25": {
          "b": 0.75,
          "k1": 1.2
        },
        "cleanupIntervalSeconds": 60,
        "stopwords": {
          "additions": null,
          "preset": "en",
          "removals": null
        }
      },
      "moduleConfig": {
        "generative-openai": {
          "deploymentId": "dc-gpt-35",
          "resourceName": "rdc-openai",
          "temperature": 0
        },
        "qna-openai": {
          "deploymentId": "dc-gpt-35",
          "resourceName": "rdc-openai",
          "temperature": 0
        },
        "text2vec-transformers": {
          "poolingStrategy": "masked_mean",
          "vectorizeClassName": true
        }
      },
      "properties": [
        {
          "dataType": [
            "text"
          ],
          "indexFilterable": true,
          "indexSearchable": true,
          "moduleConfig": {
            "text2vec-transformers": {
              "ski

In [9]:
# CLIENT.schema.delete_class('Textract')

In [14]:
uuid = CLIENT.data_object.create(
    data_object={
        "full_text": "9 April 1970= Dear Faith: Thank you for your very nice note about the Slim Critchlow record. That you, Larry, and your musical friends liked it, means a great *** deal to all of us who were involved in its production. It is true, as Jim Carr reports, that I am phasing out of running concerts and festivals. I will likely be highly involved with one more Festival (Oct.8-11 accord- ing to present plan), during my phase-out period. I am getting a little too sane (perhaps you should read \"old\") to continue trying to make a living at running events - and so, although I don't anticipate an absolutely \"straight\" job, I am looking for some relatively per- manent affiliation -most likely well out of event management. Again, thanks for your letter-- and it is very rewarding that you have such a high opinion of the record. Hope to see you soon-- Faith Petric SF Folknik 558 Clayton St. 725 @@2 San Francisco, C dif. THE and SEC was",
    },
    class_name='Textract',
    uuid='6b4b0d92-0f71-4feb-84ce-3cca10c3e4c7'
)

print(uuid)

6b4b0d92-0f71-4feb-84ce-3cca10c3e4c7


In [15]:
file_set_id = "6b4b0d92-0f71-4feb-84ce-3cca10c3e4c7"
work_id = "032944da-8e34-4812-98d5-859b927d24b4"

CLIENT.data_object.reference.add(
    from_class_name="Textract",
    from_uuid=file_set_id,
    from_property_name="hasWork",
    to_class_name="Work",
    to_uuid=work_id,
)

In [16]:
import json

data_object = CLIENT.data_object.get_by_id(
    '6b4b0d92-0f71-4feb-84ce-3cca10c3e4c7',
    class_name='Textract',
)

print(json.dumps(data_object, indent=2))

{
  "class": "Textract",
  "creationTimeUnix": 1697489364283,
  "id": "6b4b0d92-0f71-4feb-84ce-3cca10c3e4c7",
  "lastUpdateTimeUnix": 1697489474464,
  "properties": {
    "full_text": "9 April 1970= Dear Faith: Thank you for your very nice note about the Slim Critchlow record. That you, Larry, and your musical friends liked it, means a great *** deal to all of us who were involved in its production. It is true, as Jim Carr reports, that I am phasing out of running concerts and festivals. I will likely be highly involved with one more Festival (Oct.8-11 accord- ing to present plan), during my phase-out period. I am getting a little too sane (perhaps you should read \"old\") to continue trying to make a living at running events - and so, although I don't anticipate an absolutely \"straight\" job, I am looking for some relatively per- manent affiliation -most likely well out of event management. Again, thanks for your letter-- and it is very rewarding that you have such a high opinion of 